In [ ]:
import pandas as pd
import datetime
from component import *

def read_orders_from_csv(filename = './data/shenzhen_orders_0324.csv',start_date = datetime.datetime(2024,5,1),running_day = 15,full=True):
    

    end_date  = start_date + datetime.timedelta(days=running_day)
        
    fields_to_keep = ['project_id','create_time','deliver_time','order_quantity','ticket_count','pouring_type','order_id']
    type_conversions = {'project_id': str,'order_quantity': float,'ticket_count':int,'order_id':str}
    df = pd.read_csv(filename, usecols=fields_to_keep, dtype=type_conversions,parse_dates=['create_time','deliver_time'])
    
    # 将 Timestamp 转换为 datetime
    # 将 deliver_time 列转换为 datetime 类型
    df['deliver_time'] = pd.to_datetime(df['deliver_time'])
    df['create_time'] = pd.to_datetime(df['create_time'])
    
    # 不妨假设deliver_time是真实的送货时间，以此进行筛选 
    filtered_df = df[(df['deliver_time'] >= start_date) & (df['deliver_time'] <= end_date)]
    
    orders = filtered_df.to_dict(orient='records')
    for index, order in enumerate(orders, start=1):
        order['id'] = index
        
        
    if not full:
        orders = [order for order in orders if order['pouring_type']!='自卸']
        
    formed_orders = []
        
    for raw_order in orders:

        pt = raw_order['deliver_time']
        oid = raw_order['order_id']
        pid = raw_order['project_id']
        q = raw_order['order_quantity']
        count = raw_order['ticket_count']
        
        current_order = Order(oid,pid,q,count,pt) 
        formed_orders.append(current_order)
        
    return formed_orders
    
        
# 读取 CSV 文件
df = pd.read_csv('./data/shenzhen_orders_0324.csv', dtype=str)

# 提取 order_id 和 station_id，转换为字典
order_station_dict = df.set_index('order_id')['station_id'].to_dict()

def return_dispatch_list(orders:List[Order]):
    
    dispatch_list :List[Dispatch]= []
    
    for order in orders:
        order_id = order.oid
        from_sid = order_station_dict[order_id]
        
        for _ in range(order.n_need):
            new_dispatch = Dispatch(order_id,order.pid,from_sid,order.plan_arrive_time-datetime.timedelta(hours=0),from_sid)
            dispatch_list.append(new_dispatch)
        
        
    dispatch_list.sort(key=lambda x: x.dispatch_time)
    
    return dispatch_list

In [ ]:
from simulator import *






env = Environment(60,4,running_day=running_day, start_date=sd)

dispatch_list = return_dispatch_list(env.unscheduled_orders.values())
env.update_station_feature_from_dispatchs(dispatch_list)


env.continuous_running(dispatchs=dispatch_list,








In [81]:
metric_dict_new = {}

In [90]:
%load_ext autoreload
%autoreload 2
from reinforce import *
import simulator
import importlib
importlib.reload(simulator)
from simulator import *

from data_based_dispatch import *  
from heuristic_reposition import *
from copy import deepcopy

sd = datetime.datetime(2024,5,15)
running_day = 15

agent = REINFORCE(7,64,params_path='./result/model_params_0513.pth')
original_env = Environment(60,4,running_day=running_day, start_date=sd)

raw_data_dispatch = RawDataDispatch()

dispatch_list_for_fixed_need = raw_data_dispatch.return_dispatch_list(original_env.unscheduled_orders.values())
original_env.update_station_feature_from_dispatchs(dispatch_list_for_fixed_need)


reposition_funcs = [agent.take_action_max_prob]


start_truck = 60
end_truck = 70

for f in reposition_funcs:
    
    n = f.__name__ if f else 'None'
    
    
    
    while n in metric_dict_new:
        n = n + '/'
        
    print(n)
    metric_dict_new[n]= []
    
    for i in range(60,61):
        print(i)
        env=deepcopy(original_env)
        env.continuous_running(dispatchs=dispatch_list_for_fixed_need,external_method=raw_data_dispatch.external_method_for_random_need,reposition_method=f)
        
        metric_dict_new[n].append(sum(env.penalty.values())+env.cost)
        
    


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Load model parameters from ./result/model_params_0513.pth


/home/jiangrenchi/RL/reinforce.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.scorenet.load_state_dict(torch.load(params_path))


take_action_max_prob////
60


In [ ]:
{'None': [39925921.74000008],
 'LeastTruckReturn': [29928312.56000009],
 'take_action_max_prob': [47322907.5400001]}